In [20]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import featuretools as ft
import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import time

%matplotlib inline
RSEED = 50

# Load Original Features

In [2]:
feat_num = 469

In [3]:
df_total_raw = pd.read_csv('./data/features%s.csv'%(feat_num))

In [25]:
#to_drop = ['card1_count','addr2','V117','V118','V120','V122','id_25','id_21','id_26',
#           'V305','id_17','V27','V28','V88','V89','id_14','V107','V240','V241','V41','id_24','V68']
to_drop = ['addr2']

In [26]:
df_total = df_total_raw.drop(to_drop,axis=1)
#df_total = df_total_raw

In [27]:
features_train = df_total[df_total['isFraud'].notnull()]
features_test = df_total[df_total['isFraud'].isnull()]

In [28]:
labels_train = features_train['isFraud']
features_train = features_train.drop(columns = ['isFraud', 'TransactionID'])

In [29]:
features_train.shape

(590540, 466)

# Prepare model

In [30]:
categorical = ['ProductCD', 'card2', 'card3', 'card4', 'card5','card6',
              'addr1','addr2','P_email','R_email','M1','M2','M3',
              'M4','M5','M6','M7','M8','M9','DeviceType','DeviceInfo','dow','hour',
              'Device_name','Device_version','screen_width','screen_height',
               'P_email_suffix','R_email_suffix','id_30_OS','id_30_version'] 
ids = [ 'id_%s'%(i) for i in range(12,39)]
categorical = categorical + ids

In [31]:
categorical = list(set(categorical).intersection(df_total.columns))

In [32]:
features_test_new = features_test.drop(columns = ['isFraud', 'TransactionID'])

In [37]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type":'goss' ,#"gbdt"
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
          #'device' :'gpu'
          #'is_unbalance':True
          #'scale_pos_weight':9
         }

# Feature Selection

In [34]:
features_train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,card_D15_mean,card_D15_std,addr1_D15_mean,addr1_D15_std,TransactionAmt_decimal,Device_name,Device_version,is_card_freq_Device,screen_width,screen_height
0,86400,68.5,0,13926,NaN,150.0,0,142.0,0,315.0,...,0.000000,0.000000,0.000000,0.000000,500,-1,-1,False,-1,-1
1,86401,29.0,0,2755,404.0,150.0,1,102.0,0,325.0,...,0.000000,0.000000,0.000000,0.000000,0,-1,-1,False,-1,-1
2,86469,59.0,0,4663,490.0,150.0,2,166.0,1,330.0,...,2.188598,1.578656,1.420777,1.251501,0,-1,-1,False,-1,-1
3,86499,50.0,0,18132,567.0,150.0,1,117.0,1,476.0,...,0.471748,0.434842,0.581165,0.474049,0,-1,-1,False,-1,-1
4,86506,50.0,1,4497,514.0,150.0,1,102.0,0,420.0,...,NaN,NaN,NaN,NaN,0,0,0,False,0,0


In [38]:
start = time.time()
train_set = lgb.Dataset(features_train.iloc[0:472432,:], label=labels_train.values[0:472432],categorical_feature=categorical)#
valid_set = lgb.Dataset(features_train.iloc[472432:,:], label=labels_train.values[472432:],categorical_feature=categorical)#
valid_results = {}
model = lgb.train(params,train_set,num_boost_round = 10000, 
                  valid_sets = [train_set, valid_set],
                  verbose_eval=500,
                  early_stopping_rounds = 500,
                  evals_result=valid_results)
print(time.time()-start)

/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.984266	valid_1's auc: 0.919733
[1000]	training's auc: 0.997556	valid_1's auc: 0.928312
[1500]	training's auc: 0.999725	valid_1's auc: 0.929408
[2000]	training's auc: 0.999981	valid_1's auc: 0.929543
[2500]	training's auc: 0.999999	valid_1's auc: 0.929723
[3000]	training's auc: 1	valid_1's auc: 0.930053
[3500]	training's auc: 1	valid_1's auc: 0.930097
Early stopping, best iteration is:
[3208]	training's auc: 1	valid_1's auc: 0.930207
626.0921001434326


In [39]:
max(valid_results['valid_1']['auc'])

0.9302074441084627

In [40]:
lgb.plot_importance(model, max_num_features=50,figsize=(12,10))

# Train Model

In [41]:
splits = 5
folds = KFold(n_splits = splits,random_state=RSEED)
predictions = np.zeros(len(features_test_new))
ave_auc = 0

In [ ]:
for fold_num, (trn_idx, val_idx) in enumerate(folds.split(features_train.values, 
                                                          labels_train.values)):
    print("Fold {}".format(fold_num))
    train_df, y_train_df = features_train.iloc[trn_idx], labels_train.iloc[trn_idx]
    valid_df, y_valid_df = features_train.iloc[val_idx], labels_train.iloc[val_idx]
    
    trn_data = lgb.Dataset(train_df, label=y_train_df,categorical_feature=categorical)
    val_data = lgb.Dataset(valid_df, label=y_valid_df,categorical_feature=categorical)
    
    valid_results = {}
    clf = lgb.train(params,
                    trn_data,
                    10000,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds=500,
                   evals_result=valid_results)
    
    ave_auc += max(valid_results['valid_1']['auc']) / splits
    pred = clf.predict(valid_df)
    print( "  auc = ", roc_auc_score(y_valid_df, pred) )
    predictions += clf.predict(features_test_new) / splits

Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.983177	valid_1's auc: 0.905458
[1000]	training's auc: 0.997281	valid_1's auc: 0.916465
[1500]	training's auc: 0.999654	valid_1's auc: 0.91965
[2000]	training's auc: 0.999972	valid_1's auc: 0.920714
[2500]	training's auc: 0.999999	valid_1's auc: 0.92118
[3000]	training's auc: 1	valid_1's auc: 0.921808
[3500]	training's auc: 1	valid_1's auc: 0.921997
[4000]	training's auc: 1	valid_1's auc: 0.922099
Early stopping, best iteration is:
[3593]	training's auc: 1	valid_1's auc: 0.921968
  auc =  0.9219678860538212
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.984015	valid_1's auc: 0.926339
[1000]	training's auc: 0.997681	valid_1's auc: 0.935425
[1500]	training's auc: 0.999754	valid_1's auc: 0.936676
[2000]	training's auc: 0.999985	valid_1's auc: 0.936652
Early stopping, best iteration is:
[1815]	training's auc: 0.999954	valid_1's auc: 0.936873
  auc =  0.9368729599445907
Fold 2


...

Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.984652	valid_1's auc: 0.924351
[1000]	training's auc: 0.99775	valid_1's auc: 0.932155
[1500]	training's auc: 0.999755	valid_1's auc: 0.932386
Early stopping, best iteration is:
[1169]	training's auc: 0.998891	valid_1's auc: 0.932511
  auc =  0.9325105580186751
Fold 3


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.984066	valid_1's auc: 0.941507
[1000]	training's auc: 0.997652	valid_1's auc: 0.950709
[1500]	training's auc: 0.99975	valid_1's auc: 0.952392
[2000]	training's auc: 0.999984	valid_1's auc: 0.952672
Early stopping, best iteration is:
[1991]	training's auc: 0.999983	valid_1's auc: 0.952688
  auc =  0.952688253690635
Fold 4


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.984266	valid_1's auc: 0.919733
[1000]	training's auc: 0.997556	valid_1's auc: 0.928312
[1500]	training's auc: 0.999725	valid_1's auc: 0.929408
[2000]	training's auc: 0.999981	valid_1's auc: 0.929543
[2500]	training's auc: 0.999999	valid_1's auc: 0.929723
[3000]	training's auc: 1	valid_1's auc: 0.930053
[3500]	training's auc: 1	valid_1's auc: 0.930097
Early stopping, best iteration is:
[3208]	training's auc: 1	valid_1's auc: 0.930207
  auc =  0.9302074441084625


In [48]:
ave_auc

0.9353803291507821

In [32]:
ave_auc

0.9354672457640398

In [38]:
id_test = features_test['TransactionID']
submission = pd.DataFrame({'TransactionID': id_test, 
                            'isFraud': predictions})
submission.to_csv('./data/sub_feat%s_rm1_modelselect.csv'%(feat_num), index = False)